# raw_to_harmonized

In [167]:
# Import packages & setting up a connection to the ENTSOE API Client
import pandas as pd
import numpy as np

In [168]:
# Create a Local Reference
import os
current_dir = os.path.abspath("")
parent_dir = os.path.abspath(current_dir + "/../")
data_dir = os.path.abspath(parent_dir + "/data/raw/")
test_dir = os.path.abspath(parent_dir + "/data/test/")

print(current_dir)
print(parent_dir)
print(data_dir)

c:\Users\SantiTaweesamarn\Desktop\My Desktop\Data Engineering 2022\Module - Graduation Project\examproject\2. raw_to_harmonized
c:\Users\SantiTaweesamarn\Desktop\My Desktop\Data Engineering 2022\Module - Graduation Project\examproject
c:\Users\SantiTaweesamarn\Desktop\My Desktop\Data Engineering 2022\Module - Graduation Project\examproject\data\raw


In [169]:
# Read data file(s)
# import os
import glob
# import re
# import csv

# Loop through directory, Extract data from file, Transform data in pandas
for file in glob.glob(data_dir + "/*.csv"):

    # Split the path to get country code... Use later
    filename_split = file.split("_", maxsplit = 2)
    year = filename_split[0][-4:]
    country_code = filename_split[1]
    data_category = filename_split[2][:-4]

# if "generation" in file:
    with open(file) as f:
        # Read file and create DataFrame
        df = pd.read_csv(
            f,
            delimiter = ",",
            parse_dates = ["Unnamed: 0"],
        )

        # Transforming "Timestamp"-column
        df.rename(columns = {"Unnamed: 0": "Timestamp"}, inplace = True)
        df["Timestamp"] = df["Timestamp"].dt.tz_localize(None) # Replaced tz = "Europe/Stockholm" with None to get rid of "00:00:00+01:00"

        # Create a new "Country"-column and shift it to be the "first"-column
        df['Country'] = np.nan
        df['Country'] = df['Country'].replace(np.nan, filename_split[1]) # filename_split[1] returns the country code from the path
        first_column = df.pop("Country")
        df.insert(0, "Country", first_column)

        # Drop second-source columns such as that contain ".1"
        df = df[df.columns.drop(list(df.filter(regex='.1')))]

        # Drop "Marine"-column
        df = df[df.columns.drop(list(df.filter(regex='Marine')))]

        # Drop first row in DataFrame if row contains "Actual" and if filename contains "generation"
        if (data_category == "generation"):
            df = df.drop(df[df["Wind Onshore"] == "Actual Aggregated"].index)

        # Fill missing values in df
        df.fillna(int(0))

        # Load to .csv and format date
        df.to_csv(
                f"{parent_dir}/data/harmonized/{year}_{country_code}_{data_category}.csv",
                date_format = "%Y-%m-%d",
                index = False
        )